# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
import nltk
from sklearn.metrics import classification_report

In [2]:
#Load the IMDB dataset. You can load it using pandas as dataframe
dataset= pd.read_csv('IMDB Dataset.csv')
dataset.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [5]:
nltk.download('stopwords') # For stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
for columns in dataset.columns:
    
    dataset[columns] = dataset[columns].str.lower() 

stop_words=set(stopwords.words('english'))
p = re.compile(r'<.*?>')
pt=PorterStemmer()
for i in range(dataset.shape[0]):
    text1=re.sub(p,' ', dataset.iat[i,0]) #removing html
    text2=re.sub(r'http\S+', '', text1) #removing urls
    text3=re.sub(r'www\.\S+','',text2)
    text4=re.sub(r'[^A-Za-z0-9 ]+','', text3) #removing non alphanumeric characters
    wordtokens=word_tokenize(text4)
    filtertokens=[]
    for token in wordtokens:
        if token not in stop_words:     #removing stopwords
            filtertokens.append(token)
    
    wordlist=[]
    for word in filtertokens:
        wordlist.append(pt.stem(word))    #performing word stemming
    string=''
    for word in wordlist:
        string=string+word+' '
    
    dataset.iat[i,0]=string.strip()
        
    

dataset.head()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,positive


In [6]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
total=0
maxlen=0
index=0
totalpos=0
pos=0
totalneg=0
neg=0
vocab=[]
for i in range(dataset.shape[0]):
    for word in dataset.iat[i,0].split(' '):
        if word not in vocab:
            vocab.append(word)
    total=total + len(dataset.iat[i,0].split(' '))
    if len(dataset.iat[i,0].split(' '))>maxlen:
        maxlen=len(dataset.iat[i,0].split(' '))
        index=i
    if dataset.iat[i,1]=='positive':
        totalpos=totalpos+len(dataset.iat[i,0].split(' '))
        pos=pos+1
    if dataset.iat[i,1]=='negative':
        totalneg=totalneg+len(dataset.iat[i,0].split(' '))
        neg=neg+1

print('average review length :',total/dataset.shape[0])
print('maximum review length : ',maxlen)
print('average length of positive review  :',totalpos/pos)
print('No of positive reviews :',pos)
print('average length of negative review  :',totalneg/neg)
print('No of negative reviews :',neg)
print('Vocabulary size :',len(vocab))






average review length : 119.68862
maximum review length :  1437
average length of positive review  : 121.02932
No of positive reviews : 25000
average length of negative review  : 118.34792
No of negative reviews : 25000
Vocabulary size : 154450


# Naive Bayes classifier

In [7]:
# get reviews column from df
reviews = dataset['review']

# get labels column from df
labels = dataset['sentiment']

In [8]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoded_labels)
print(encoder.classes_)

[1 1 1 ... 0 0 0]
['negative' 'positive']


In [9]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.20, random_state=1, stratify=encoded_labels)
# train_sentences, test_sentences, train_labels, test_labels


Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words
'''
max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(Sentence_list)
'''
maxfeatures=3000
vec = CountVectorizer(max_features = maxfeatures)
X = vec.fit_transform(train_sentences)
wordsdict=vec.get_feature_names()

print(X)
print(X.nonzero())


  (0, 2212)	1
  (0, 592)	1
  (0, 2494)	1
  (0, 755)	3
  (0, 941)	1
  (0, 1751)	15
  (0, 2074)	1
  (0, 2852)	2
  (0, 2537)	1
  (0, 2922)	1
  (0, 651)	1
  (0, 2565)	6
  (0, 2255)	1
  (0, 1717)	1
  (0, 1897)	1
  (0, 2513)	1
  (0, 1025)	1
  (0, 1619)	2
  (0, 1258)	1
  (0, 798)	1
  (0, 1561)	8
  (0, 2900)	1
  (0, 766)	4
  (0, 165)	4
  (0, 890)	1
  :	:
  (39999, 2223)	1
  (39999, 1395)	1
  (39999, 2966)	2
  (39999, 1626)	1
  (39999, 291)	1
  (39999, 892)	1
  (39999, 1916)	1
  (39999, 2466)	4
  (39999, 1578)	1
  (39999, 2569)	1
  (39999, 1327)	1
  (39999, 1018)	1
  (39999, 839)	1
  (39999, 1658)	1
  (39999, 2704)	2
  (39999, 372)	1
  (39999, 1338)	1
  (39999, 1205)	1
  (39999, 2775)	1
  (39999, 1562)	1
  (39999, 750)	1
  (39999, 2426)	1
  (39999, 710)	1
  (39999, 628)	1
  (39999, 1443)	1
(array([    0,     0,     0, ..., 39999, 39999, 39999], dtype=int32), array([2212,  592, 2494, ...,  710,  628, 1443], dtype=int32))


In [11]:
# Use laplace smoothing for words in test set not present in vocab of train set
from scipy.sparse import csr_matrix, find
trainindex=find(X)[0]
wordnum=find(X)[1]
value=find(X)[2]


probdictpos={}
for i in wordsdict:
    probdictpos[i]=0
    
probdictneg={}
for i in wordsdict:
    probdictneg[i]=0
ncountpos=0
ncountneg=0
for i in range(len(trainindex)):
    if train_labels[trainindex[i]]==1:
        probdictpos[wordsdict[wordnum[i]]]=probdictpos[wordsdict[wordnum[i]]] + value[i]
        ncountpos=ncountpos+value[i]
    elif train_labels[trainindex[i]]==0:
        probdictneg[wordsdict[wordnum[i]]]=probdictneg[wordsdict[wordnum[i]]]+value[i]
        ncountneg=ncountneg+value[i]
        
for key,value in probdictpos.items():
    probdictpos[key]=(value+1)/(ncountpos + maxfeatures)
for key,value in probdictneg.items():
    probdictneg[key]=(value+1)/(ncountneg + maxfeatures)
        



In [12]:

# Build the model. Don't use the model from sklearn
def modelfunc(sentence):
    pos=0
    neg=0
    for word in sentence.split(' '):
        
        if word in probdictpos.keys():
            
            pos=pos+np.log(probdictpos[word])
        else:
            pos=pos+np.log(1/(ncountpos+maxfeatures))
        
        if word in probdictneg.keys():
            neg=neg+np.log(probdictneg[word])
        else:
            neg=neg+np.log(1/(ncountneg+maxfeatures))
        
  
    if pos>neg:
        return 1
    else:
        return 0
        
        
        
        
    



In [13]:
# Test the model on test set and report Accuracy

count=0
#print(test_sentences.iat[0])
#print(type(test_sentences))
for i in range(len(test_sentences)):
    #print(i)
    label=modelfunc(test_sentences.iat[i])
    #print(label,test_labels[i])
    if label==test_labels[i]:
        count=count+1
        
print('Accuracy :',count/len(test_sentences))
    



Accuracy : 0.8428


# *LSTM* based Classifier

Use the above train and test splits.

In [15]:
# Hyperparameters of the model
vocab_size = len(vocab) 
oov_tok = '<OOK>'
embedding_dim = 100
max_length = maxlen # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [16]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [17]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1437, 100)         15445000  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 15,532,601
Trainable params: 15,532,601
Non-trainable params: 0
_________________________________________________________________


In [18]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 529s 471ms/step - loss: 0.3831 - accuracy: 0.8317 - val_loss: 0.2792 - val_accuracy: 0.8945
Epoch 2/5
1125/1125 [==============================] - 529s 471ms/step - loss: 0.2477 - accuracy: 0.9055 - val_loss: 0.4803 - val_accuracy: 0.7673
Epoch 3/5
1125/1125 [==============================] - 528s 470ms/step - loss: 0.1851 - accuracy: 0.9334 - val_loss: 0.3043 - val_accuracy: 0.8823
Epoch 4/5
1125/1125 [==============================] - 527s 468ms/step - loss: 0.0960 - accuracy: 0.9678 - val_loss: 0.3220 - val_accuracy: 0.8885
Epoch 5/5
1125/1125 [==============================] - 525s 466ms/step - loss: 0.0544 - accuracy: 0.9826 - val_loss: 0.4194 - val_accuracy: 0.8850


In [20]:
# Calculate accuracy on Test data
'''
prediction = model.predict(test_padded)

'''
# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
output=prediction>=0.5


# Accuracy : one can use classification_report from sklearn
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(test_labels, output)
print('Accuracy ',accuracy)

Accuracy  0.8785


## Get predictions for random examples

In [22]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences =  tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities

outv=model.predict(padded)
print(outv)


# Get labels based on probability 1 if p>= 0.5 else 0

output2= outv>=0.5

for i in range(3):
   
    if output2[i][0]==True:
        print(sentence[i],':  positive')
    else:
        print(sentence[i],':  negative')



[[0.9087079 ]
 [0.44581446]
 [0.10172906]]
The movie was very touching and heart whelming :  positive
I have never seen a terrible movie like this :  negative
the movie plot is terrible but it had good acting :  negative
